## 訓練前の準備

モデルをインスタンス化し、モデルとオプティマイザを初期化します。

次にDatasetとDataLoaderインスタンスの初期化をする。datasetで要素をランダムかし、Dataloaderでデータをアプリケーションへ供給する。


In [ ]:
class LunaDataset(Dataset):
    def __init__(self,
                 val_stride=0,
                 isValSet_bool=None,
                 series_uid=None,
            ):
        self.candidateInfo_list = copy.copy(getCandidateInfoList())

        if series_uid:
            self.candidateInfo_list = [
                x for x in self.candidateInfo_list if x.series_uid == series_uid
            ]

        if isValSet_bool:
            assert val_stride > 0, val_stride
            self.candidateInfo_list = self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
        elif val_stride > 0:
            del self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
        random.shuffle(self.candidateInfo_list)
#
#
#Datasetではリストをランダム化している

アプリケーションでの初期化

cudaが使用可能かでdeviceを、GPUが二つ以上（１以上）ならnn.DataParallel(model)として作業を分割できる。
結果の集計、パラメータ更新の同期等をとってくれるため、特段意識する必要はない。

modelのパラメータをGPUへ転送するときは必ずオプティマイザをインスタンス化する前にモデルをGPUへ転送する。そうしないとGPU内のコピーされたパラメータではなく、
CPUのパラメータを最適化しようとする。

In [ ]:
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

        parser = argparse.ArgumentParser()
        parser.add_argument('--num-workers',
            help='Number of worker processes for background data loading',
            default=8,
            type=int,
        )
        
        #
        #
        #
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.model = self.initModel()
        self.optimizer = self.initOptimizer()

    def initModel(self):
        model = LunaModel()
        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1:
                model = nn.DataParallel(model)
            model = model.to(self.device)
        return model

    def initOptimizer(self):
        return SGD(self.model.parameters(), lr=0.001, momentum=0.99)
        return Adam(self.model.parameters())
        
        #
        #
        #

## オプティマイザ

まずSGDから始めるのが妥当、学習率は0.001、モーメンタムは0.9が多くのプロジェクトでうまく機能することが知られている。
試してみてうまくいかないときに学習率を0.01や0.0001を試してみるのも良い。

ハイパーパラメータのチューニングはプロジェクトの多くの問題に対処した後で検討すればよい。

## データローダー

バッチ化作業を行う。DatasetクラスにてCT画像からテンソルへ変換されているため、データを渡すだけで良い。

sys.argvは、Pythonスクリプトに渡された、コマンドライン引数のリストです。

sys.argv[0]はスクリプトの名前です。
sys.argv[1]はコマンドライン引数の一番目

In [ ]:
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

        parser.add_argument('--batch-size',
            help='Batch size to use for training',
            default=32,
            type=int,
                           )
        #
        #
        #
        
        self.cli_args = parser.parse_args(sys_argv)
        #
        #
        #

In [ ]:
 def initTrainDl(self):
        train_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=False,
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        train_dl = DataLoader(
            train_ds,
            batch_size=batch_size,
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda,
        )

        return train_dl

In [ ]:
 def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl()
        
        
        #
        #
        #

## モデル構成

畳み込みから全結合層へ入力うするブロックを作成して
このブロックを重ねてディープネットワークとする。

In [ ]:
class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu2 = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool3d(2, 2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)


## モデル全体

ブロックを四回重ねて、全結合層へ

出力はソフトマックスを通す前のロジットと確率の二種類である。

訓練中に損失関数nn.crosentropyを用いる場合はロジットを使用。
実際にサンプルをクラス分類するときは確率を用いる。

In [ ]:
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1)

        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)

        self.head_linear = nn.Linear(1152, 2)
        self.head_softmax = nn.Softmax(dim=1)

        self._init_weights()


    def forward(self, input_batch):
        bn_output = self.tail_batchnorm(input_batch)

        block_out = self.block1(bn_output)
        block_out = self.block2(block_out)
        block_out = self.block3(block_out)
        block_out = self.block4(block_out)

        conv_flat = block_out.view(
            block_out.size(0),
            -1,
        )
        linear_output = self.head_linear(conv_flat)

        return linear_output, self.head_softmax(linear_output)


ネットワークの重みの初期化

全ての重みが1より大きい又は全ての重みが1より小さいと出力の値がとても大きくなるか、0となる。
このようにならないために層の出力は適切な範囲となるようにする。

下の中身の理解は不要とのこと


In [ ]:
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()
    #
    #
    #

    def _init_weights(self):
        for m in self.modules():
            if type(m) in {
                nn.Linear,
                nn.Conv3d,
                nn.Conv2d,
                nn.ConvTranspose2d,
                nn.ConvTranspose3d,
            }:
                nn.init.kaiming_normal_(
                    m.weight.data, a=0, mode='fan_out', nonlinearity='relu',
                )
                if m.bias is not None:
                    fan_in, fan_out = \
                        nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)

## モデルの訓練と検証

In [ ]:
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

    #
    #
    #
    def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl()

        for epoch_ndx in range(1, self.cli_args.epochs + 1):

            log.info("Epoch {} of {}, {}/{} batches of size {}*{}".format(
                epoch_ndx,
                self.cli_args.epochs,
                len(train_dl),
                len(val_dl),
                self.cli_args.batch_size,
                (torch.cuda.device_count() if self.use_cuda else 1),
            ))

            trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
            self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

            valMetrics_t = self.doValidation(epoch_ndx, val_dl)
            self.logMetrics(epoch_ndx, 'val', valMetrics_t)

        if hasattr(self, 'trn_writer'):
            self.trn_writer.close()
            self.val_writer.close()


    def doTraining(self, epoch_ndx, train_dl):
        self.model.train()
        #（METRICS_SIZE = 3） * データの総数のマトリクスを作成
        trnMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(train_dl.dataset),
            device=self.device,
        )

        #終了時間をしるためのもの
        batch_iter = enumerateWithEstimate(
            train_dl,
            "E{} Training".format(epoch_ndx),
            start_ndx=train_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.optimizer.zero_grad()

            loss_var = self.computeBatchLoss(
                batch_ndx,
                batch_tup,
                train_dl.batch_size,
                trnMetrics_g
            )

            #実際にモデルの重み更新
            loss_var.backward()
            self.optimizer.step()


        self.totalTrainingSamples_count += len(train_dl.dataset)

        return trnMetrics_g.to('cpu')



doTrainingのtrnMetrics_g に評価指標が保管される。

##　評価指標を詳細に保存する

訓練ループでも検証ループでも同様に下記のloss関数を使う。

テンソルをGPUへ
モデルを通し損失を計算する

そのlossの平均はバッチの損失であるが、間で処理を可能にしている。

In [ ]:
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
        input_t, label_t, _series_list, _center_list = batch_tup

        input_g = input_t.to(self.device, non_blocking=True)
        label_g = label_t.to(self.device, non_blocking=True)

        logits_g, probability_g = self.model(input_g)

        loss_func = nn.CrossEntropyLoss(reduction='none')
        loss_g = loss_func(
            logits_g,
            label_g[:,1],
        )
        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + label_t.size(0)

        #1.答え
        metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = \
            label_g[:,1].detach()
        #2.正しいクラスの確率
        metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = \
            probability_g[:,1].detach()
        #3.損失
        metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = \
            loss_g.detach()

        return loss_g.mean()


metrics_gにはラベルと予測確率、LOSSが入る

In [ ]:
# Used for computeBatchLoss and logMetrics to index into metrics_t/metrics_a
METRICS_LABEL_NDX=0
METRICS_PRED_NDX=1
METRICS_LOSS_NDX=2
METRICS_SIZE = 3


        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + label_t.size(0)

        metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = \
            label_g[:,1].detach()
        metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = \
            probability_g[:,1].detach()
        metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = \
            loss_g.detach()

## 検証ル―プ

検証ではデータの読み込みのみで損失の計算はない。
with torch no_grad()によって勾配の計算もない。

In [ ]:
 def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl()

        for epoch_ndx in range(1, self.cli_args.epochs + 1):
            
            #
            #
            #
            valMetrics_t = self.doValidation(epoch_ndx, val_dl)
            self.logMetrics(epoch_ndx, 'val', valMetrics_t)

In [ ]:
正しい検証のためにパラメータの更新は必ず止める
computeBatchLossの戻り値のLOSSも不要（記録のみ追加）、オプティマイザ―も使わない。

In [ ]:
def doValidation(self, epoch_ndx, val_dl):
        with torch.no_grad():
            self.model.eval()
            valMetrics_g = torch.zeros(
                METRICS_SIZE,
                len(val_dl.dataset),
                device=self.device,
            )

            batch_iter = enumerateWithEstimate(
                val_dl,
                "E{} Validation ".format(epoch_ndx),
                start_ndx=val_dl.num_workers,
            )
            for batch_ndx, batch_tup in batch_iter:
                self.computeBatchLoss(
                    batch_ndx, batch_tup, val_dl.batch_size, valMetrics_g)

        return valMetrics_g.to('cpu')

## モデルの評価指標を出力

各エポックごとに評価指標のログを出力する。

訓練が想定通りに進んでいない（収束しない場合）に訓練を中止するためにも必要

trn_metricsとval_metricsには各エポックごとの結果が記録されている。
この二つからクラスごとの正解率と損失を確認する。


epoch_ndxは何エポックの時なのか、mode_strは訓練か検証か、metricsはtreかvakのmetricsを入れる。

In [ ]:
def logMetrics(
            self,
            epoch_ndx,
            mode_str,
            metrics_t,
            classificationThreshold=0.5,
    ):

metericsの結果をクラス（結節か非結節かのラベル）ごとに分けるため、マスクを作成する。

In [16]:
a=torch.ones([1,2], dtype=torch.bool)
a

tensor([[True, True]])

In [18]:
b =~a
b

tensor([[False, False]])

In [ ]:
def logMetrics(
            self,
            epoch_ndx,
            mode_str,
            metrics_t,
            classificationThreshold=0.5,
        #
        #
        #
        negLabel_mask = metrics_t[METRICS_LABEL_NDX] <= classificationThreshold
        negPred_mask = metrics_t[METRICS_PRED_NDX] <= classificationThreshold

        posLabel_mask = ~negLabel_mask
        posPred_mask = ~negPred_mask

Thresholdの0.5以下を非結節、0.5以上を結節と予測したとし、booleanのマスクをつくる

[METRICS_LABEL_DX]はラベルなので0.0か1.0の集合である

In [ ]:
def logMetrics(
            self,
            epoch_ndx,
            mode_str,
            metrics_t,
            classificationThreshold=0.5,        
    　　#
        #
        #
        neg_count = int(negLabel_mask.sum())
        pos_count = int(posLabel_mask.sum())

        neg_correct = int((negLabel_mask & negPred_mask).sum())
        pos_correct = int((posLabel_mask & posPred_mask).sum())

        metrics_dict = {}
        metrics_dict['loss/all'] = \
            metrics_t[METRICS_LOSS_NDX].mean()
        metrics_dict['loss/neg'] = \
            metrics_t[METRICS_LOSS_NDX, negLabel_mask].mean()
        metrics_dict['loss/pos'] = \
            metrics_t[METRICS_LOSS_NDX, posLabel_mask].mean()

        metrics_dict['correct/all'] = (pos_correct + neg_correct) \
            / np.float32(metrics_t.shape[1]) * 100
        metrics_dict['correct/neg'] = neg_correct / np.float32(neg_count) * 100
        metrics_dict['correct/pos'] = pos_correct / np.float32(pos_count) * 100

loss/all に　エポックごとの平均した損失

negLabel_maskを使っているので

loss/neg に　実際に結節でないいサンプルの損失

loss/pos に　実際に結節であるサンプルの損失

が格納される

各correctにはラベルマスクも予測マスクも同じである数の合計があり、
それぞれの正解率も辞書に格納される。（metrics_tはデータ総数）

## log

ライブラリ側（使われる側）では、

先頭にlogger = logging.getLogger(__name__)を指定してモジュールごとにロガーを作成する
ログを出力したい場所にlogger.debug(‘メッセージ’)のように書く


ユーザ側（使う側）では、

細かな出力方法はユーザー側で設定する

In [19]:
#taining.pyの初め

log = logging.getLogger(__name__)
# log.setLevel(logging.WARN)
log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

レベルを指定して、それ以上のレベルのログのみ表示される。

下は表示したい処理の時にlog.infoでinfoレベルのログを表示する。

In [ ]:
def logMetrics(
            self,
    #
    #
    #
        log.info(
            ("E{} {:8} {loss/all:.4f} loss, "
                 + "{correct/all:-5.1f}% correct, "
            ).format(
                epoch_ndx,
                mode_str,
                **metrics_dict,
            )
        )
        log.info(
            ("E{} {:8} {loss/neg:.4f} loss, "
                 + "{correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_neg',
                neg_correct=neg_correct,
                neg_count=neg_count,
                **metrics_dict,
            )
        )
        log.info(
            ("E{} {:8} {loss/pos:.4f} loss, "
                 + "{correct/pos:-5.1f}% correct ({pos_correct:} of {pos_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_pos',
                pos_correct=pos_correct,
                pos_count=pos_count,
                **metrics_dict,
            )
        )

## 訓練の実施

訓練中に研鑽リソースが想定通り使われているか確認したいとき（データの読み込みか計算のどっちがボトルネックになっているのか）

training 16/7750, done のようなメッセージが表示されたら
topコマンドとnvidia-smiコマンドを使用

Pythonのワーカープロセスが80以上のCPUを使っていればキャッシュの必要があるかもしれない（こんかいは）
nvidia_smiでGPU-Util が80以上なら効率よくGPUが使われている。

## 訓練データについて

実際のデータよりも訓練や検証データが少ない場合は本当にデータが存在するのか確認した方が良い。

ls -1 "データフォルダのpath/*"  \  wc -l

## 正解率99.7%は正しいのか